<a href="https://colab.research.google.com/github/Naga-SDonepudi/PyTorch_HandsOn/blob/main/6_splitting_code_to_reusable_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Converting code in cells to Reusable scripts
* This is termed as going modular using python scripts
* The code in cells performing specific functionality like data loading, model builidng and training, model eval, save can be made as Python scripts

## 1. Data

In [1]:
import os
import zipfile
from pathlib import Path
import requests

# Setting a path for data folder
data_path = Path("data/")
img_path = data_path / "pizza_steak_sushi"

if img_path.is_dir():
  print(f"{img_path} directory was already there")
else:
  print(f"Creating a {img_path} directory")
  img_path.mkdir(parents=True, exist_ok=True)

# Downloading
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/Naga-SDonepudi/PyTorch_HandsOn/raw/main/pizza_steak_sushi.zip")
  print("Downloaded the zip file")
  f.write(request.content)

# Unzipping the data file
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping the downloaded zip file")
  zip_ref.extractall(img_path)
os.remove(data_path / "pizza_steak_sushi.zip")

Creating a data/pizza_steak_sushi directory
Downloaded the zip file
Unzipping the downloaded zip file


In [2]:
## Setting a path for training and testing data
train_dir = img_path / "train"
test_dir = img_path / "test"
train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

### 2. Data to PyTorch Datsets and DataLoaders

In [3]:
from torchvision import datasets, transforms

# Transform
data_transform = transforms.Compose([transforms.Resize((64, 64)),
                                     transforms.ToTensor()])

# ImageFolder for creating datasets
train_data = datasets.ImageFolder(root=train_dir,
                                  transform=data_transform,
                                  target_transform=None)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)
train_data, test_data

(Dataset ImageFolder
     Number of datapoints: 225
     Root location: data/pizza_steak_sushi/train
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
                ToTensor()
            ),
 Dataset ImageFolder
     Number of datapoints: 75
     Root location: data/pizza_steak_sushi/test
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
                ToTensor()
            ))

In [4]:
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [5]:
## data to dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1,
                              num_workers=1,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7f8ed9015ca0>,
 <torch.utils.data.dataloader.DataLoader at 0x7f8ed93e7fb0>)

In [6]:
# Directory for storing scripts (modular)
import os
os.makedirs("modular_scripts")

## Script Mode using %%writefile

In [18]:
%%writefile modular_scripts/data_setup_script.py
"""
Consists of functionality for DataLoaders for classifying image
"""

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS):
  """
    Creates training and test dataloaders by taking in a train and test directory, turns them to datasets using ImageFolder and then to PyTorch DataLoaders
  Args:
    train_dir and test_dir are paths for train and test directories
    transforms on train and test data
    batch_size means number of samples per batch in each of DataLoaders
    num_workers means num of workers per dataloader
  Returns:
     Returns a tuple of train_dataloader, test_dataloader, class_names (list of target classes)
  """

  # ImageFolder for creating datasets
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(root=test_dir, transform=transform)

  # Class names
  class_names = train_data.classes

  # Images to dataloaders
  train_dataloader = DataLoader(dataset=train_data,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=True,
                                pin_memory=True)

  test_dataloader = DataLoader(dataset=test_data,
                               batch_size=batch_size,
                               num_workers=num_workers,
                               shuffle=False,
                               pin_memory=True)

  return train_dataloader, test_dataloader, class_names

Writing modular_scripts/data_setup_script.py


### Importing the .py file(modular script)

In [19]:
from modular_scripts import data_setup_script

In [21]:
train_dataloader, test_dataloader, class_names = data_setup_script.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=data_transform,
                                                                               batch_size=32)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f8ed8e59be0>,
 ['pizza', 'steak', 'sushi'])

### 3. Model building and turning to script mode

In [31]:
%%writefile modular_scripts/model_script.py
import torch
from torch import nn
"""
PyTorch model code """
class TinyVGG(nn.Module):
  def __init__(self, input_shape: int,
               hidden_units: int,
               output_shape: int) -> None:
    super().__init__()

    # Convolution block 1
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )

    # Convolution block 2
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*16*16,
                  out_features=output_shape)
    )

  def forward(self, x):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)

    return x

Overwriting modular_scripts/model_script.py


In [33]:
## peforming forward from model_script
from modular_scripts import model_script
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiating
torch.manual_seed(42)
model_1 = model_script.TinyVGG(input_shape=3,
                               hidden_units=10,
                               output_shape=len(class_names)).to(device)
model_1

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2560, out_features=3, bias=True)
  )
)

### 4. train_mode and test_mode functions to train() step

In [34]:
from typing import Tuple
## Creating training steps by starting off with defining a function
def train_mode(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device=device) -> Tuple[float, float]:

  model.train()

  # Setting up evaluation metrics
  train_loss, train_acc = 0,0

  # Loop through dataloader databatch
  for batch, (X,y) in enumerate(dataloader):

    # Sending data to target device
    X,y = X.to(device), y.to(device)

    ## 1. Performing forward pass
    y_pred = model(X)

    ## 2. Loss calculation and accumulate to train_loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    ## 3. Optimizer zero grad
    optimizer.zero_grad()

    ## 4. Backward loss propagarion
    loss.backward()

    ## 5. Optimization
    optimizer.step()

    ## Calculating accuracy
    y_pred_class = y_pred.argmax(dim=1)
    train_acc += (y_pred_class==y).sum().item() / len(y_pred)

  ## Adjusting metrics for avg loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [35]:
## Testing loop
def test_mode(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):

  # Eval mode
  model.eval()

  # Test loss and tess accuracy metrics values
  test_loss, test_acc = 0,0

  # Inference mode
  with torch.inference_mode():

    ## Loop through dl
    for batch, (X, y) in enumerate(dataloader):

      # Data to target device
      X, y = X.to(device), y.to(device)

      ## Forward Pass
      test_pred = model(X)

      ## Loss
      loss = loss_fn(test_pred, y)
      test_loss += loss.item()

      # Accuracy
      test_pred_labels = test_pred.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))

  ## Adjusting metrics
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

In [36]:
## Combining both to train step
from tqdm.auto import tqdm

## Creating a function for train
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module=nn.CrossEntropyLoss(),
          epochs: int=5,
          device=device):

  ## Creating an empty dictionary
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  # Looping through the train and tetsing steps
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_mode(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)

    test_loss, test_acc = test_mode(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    ## Printing out whats goin on
    print(f"Epoch: {epoch} | Training Loss: {train_loss:.4f} | Training Accuracy: {train_acc:.2f}% | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

    ## Results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results

### 4. Turning this to a script (train_mode and test_mode functions to train() step then to a .py script)

In [42]:
%%writefile modular_scripts/engine_script.py
""" Contains functions for training and testing a model """

from typing import Tuple, Dict, List
import torch
from torch import nn
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

## Train Mode Function
def train_mode(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device=device) -> Tuple[float, float]:

  model.train()

  # Setting up evaluation metrics
  train_loss, train_acc = 0,0

  # Loop through dataloader databatch
  for batch, (X,y) in enumerate(dataloader):

    # Sending data to target device
    X,y = X.to(device), y.to(device)

    ## 1. Performing forward pass
    y_pred = model(X)

    ## 2. Loss calculation and accumulate to train_loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    ## 3. Optimizer zero grad
    optimizer.zero_grad()

    ## 4. Backward loss propagarion
    loss.backward()

    ## 5. Optimization
    optimizer.step()

    ## Calculating accuracy
    y_pred_class = y_pred.argmax(dim=1)
    train_acc += (y_pred_class==y).sum().item() / len(y_pred)

  ## Adjusting metrics for avg loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc


## Testing loop
def test_mode(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):

  # Eval mode
  model.eval()

  # Test loss and tess accuracy metrics values
  test_loss, test_acc = 0,0

  # Inference mode
  with torch.inference_mode():

    ## Loop through dl
    for batch, (X, y) in enumerate(dataloader):

      # Data to target device
      X, y = X.to(device), y.to(device)

      ## Forward Pass
      test_pred = model(X)

      ## Loss
      loss = loss_fn(test_pred, y)
      test_loss += loss.item()

      # Accuracy
      test_pred_labels = test_pred.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))

  ## Adjusting metrics
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

## Combining both to train step
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module=nn.CrossEntropyLoss(),
          epochs: int=5,
          device=device):

  ## Creating an empty dictionary
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  # Looping through the train and tetsing steps
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_mode(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)

    test_loss, test_acc = test_mode(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    ## Printing out whats goin on
    print(f"Epoch: {epoch} | Training Loss: {train_loss:.4f} | Training Accuracy: {train_acc:.2f}% | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

    ## Results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results

Overwriting modular_scripts/engine_script.py


### 5. Utility functions file script

In [44]:
%%writefile modular_scripts/utils_script.py
"""
Contains  utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing modular_scripts/utils_script.py


In [ ]:
### 6. Turning

In [45]:
%%writefile modular_scripts/train_script.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup_script, engine_script, model_script, utils_script

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 6
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup_script.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_script.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine_script.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils_script.save_model(model=model,
                 target_dir="models",
                 model_name="06_modular_script_mode_tinyvgg_model.pth")

Writing modular_scripts/train_script.py


In [47]:
!python modular_scripts/train_script.py

  0% 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch: 0 | Training Loss: 1.1050 | Training Accuracy: 0.39% | Test Loss: 1.1468 | Test Accuracy: 0.26%
 17% 1/6 [00:02<00:11,  2.35s/it]Epoch: 1 | Training Loss: 1.1304 | Training Accuracy: 0.30% | Test Loss: 1.1033 | Test Accuracy: 0.26%
 33% 2/6 [00:04<00:08,  2.07s/it]Epoch: 2 | Training Loss: 1.0912 | Training Accuracy: 0.31% | Test Loss: 1.1010 | Test Accuracy: 0.34%
 50% 3/6 [00:06<00:07,  2.39s/it]Epoch: 3 | Training Loss: 1.0944 | Training Accuracy: 0.32% | Test Loss: 1.1010 | Test Accuracy: 0.20%
 67% 4/6 [00:08<00:04,  2.11s/it]Epoch: 4 | Training Loss: 1.0629 | Training Accuracy: 0.48% | Test Loss: 1.0834 | Test Accuracy: 0.31%
 83% 5/6 [00:10<00:01,  1.99s/it]Epoch: 5 | Training Loss: 1.0586 | Training Accuracy: 0.38% | Test Loss: 